In [1]:
import pandas as pd
import numpy as np
import pickle

**Section 1: Data Cleaning**

Like with any model development process, we will begin by sifting through our dataset and extracting only the required features.

In [5]:
elo_df = pd.read_csv("./mlb_elo.csv")

In [7]:
elo_df = elo_df.loc[elo_df['season'] >= 2000]
elo_df = elo_df.loc[elo_df['season'] <= 2020]
# elo_df = elo_df[~elo_df['pitcher1'].str.contains(r'[0-9]')]
# elo_df = elo_df[~elo_df['pitcher2'].str.contains(r'[0-9]')]

elo_df['date']= pd.to_datetime(elo_df['date'])

elo_df.head()

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
2430,2020-10-27,2020,1,w,LAD,TBD,1607.757915,1564.177984,0.582851,0.417149,...,54.055790,54.534448,7.995742,6.040024,0.594547,0.405453,1612.374351,1561.153482,3.0,1.0
2431,2020-10-25,2020,1,w,TBD,LAD,1566.449752,1605.486147,0.425652,0.574348,...,52.966933,57.214954,-1.826734,23.114096,0.368716,0.631284,1562.804875,1610.722958,2.0,4.0
2432,2020-10-24,2020,1,w,TBD,LAD,1563.862158,1608.073741,0.415971,0.584029,...,52.636552,50.974099,-3.959517,-6.204884,0.411161,0.588839,1564.204361,1609.323472,8.0,7.0
2433,2020-10-23,2020,1,w,TBD,LAD,1567.099442,1604.836456,0.428091,0.571909,...,57.971157,55.937980,20.546588,18.098422,0.420376,0.579624,1562.243577,1611.284256,2.0,6.0
2434,2020-10-21,2020,1,w,LAD,TBD,1608.352409,1563.583490,0.585068,0.414932,...,54.975487,54.421564,13.013222,4.009710,0.611092,0.388908,1608.914363,1564.613470,4.0,6.0


In [8]:
elo_df.to_csv( "elo_cleaned.csv")

In [ ]:
elo1_pre = 1531.8233671069
elo2_pre = 1456.6148797361
elo1_post = 1529.9288796768
elo2_post = 1458.5093671663

trans_rating1 = 10**(elo1_pre / 400.0)
trans_rating2 = 10**(elo2_pre / 400.0)

exp_score1 = elo1_pre / (elo1_pre + elo2_pre)
exp_score2 = elo2_pre / (elo1_pre + elo2_pre)

win_loss1 = 0
win_loss2 = 1

# calculate k
k1 = (elo1_post - elo1_pre) / (win_loss1 - exp_score1)
k2 = (elo2_post - elo2_pre) / (win_loss2 - exp_score2)

print("K1: " + str(k1))
print("K2: " + str(k2))

# 4.64
# 4.60
# 5.10



In [ ]:
def elo_k(point_diff, elo_diff):
    k = 32

    if point_diff>0:
        multiplier=(point_diff)**(0.5)/(10+0.01*(elo_diff))
    else:
        multiplier=(-point_diff)**(0.5)/(10+0.01*(-elo_diff))
    return k*multiplier

print(elo_k(-1, elo1_pre - elo2_pre))

In [2]:
loaded_something = pickle.load(open('2019_data_june23.pickle', 'rb'))
loaded_something.head()

,home_win,home_score,away_score,date,lookback_days,home_team,away_team,home_pct,away_pct,home_streak,...,awaypen_bb,awaypen_k,awaypen_at_bats,awaypen_doub,awaypen_trip,awaypen_hr,awaypen_era,awaypen_ba,awaypen_slg,awaypen_obp
0,0,1.0,9.0,2019-04-10,10,CHW,TB,0.300000,0.750000,0.375000,...,2.679389,9.687023,32.977099,0.824427,0.000000,0.412214,1.442748,0.200000,0.262,0.260
1,0,1.0,15.0,2019-04-10,10,PHI,WSN,0.700000,0.500000,0.625000,...,5.934066,9.494505,38.274725,2.373626,0.000000,1.780220,8.604396,0.317829,0.519,0.409
2,1,7.0,2.0,2019-04-10,10,STL,LAD,0.545455,0.666667,0.625000,...,3.469274,7.843575,34.541899,1.810056,0.603352,1.357542,4.675978,0.244541,0.450,0.313
3,0,1.0,3.0,2019-04-10,10,SF,SD,0.333333,0.583333,0.333333,...,3.258621,10.474138,34.448276,2.793103,0.000000,0.931034,3.724138,0.243243,0.405,0.309
4,1,9.0,6.0,2019-04-10,10,NYM,MIN,0.600000,0.666667,0.500000,...,5.400000,5.737500,33.075000,1.350000,0.000000,1.012500,3.375000,0.234694,0.367,0.342


In [49]:
game_log_df = pd.read_csv("./game_logs_00_20.csv")
game_log_df['date']= pd.to_datetime(game_log_df['date'])
game_log_df['h_team'] = game_log_df['h_team'].astype(str)
game_log_df['a_team'] = game_log_df['a_team'].astype(str)

# Handle alternate team abbreviation
game_log_df["h_team"].replace({"NYN": "NYM", "CHN": "CHC", "FLO": "FLA", "MON": "WSN", "SLN": "STL", "KCA": "KCR", "TBA": "TBD", "SDN": "SDP", "NYA": "NYY", "LAN": "LAD", "WAS": "WSN", "MIA": "FLA", "CHA": "CHW", "SFN": "SFG", "nan": float("NaN")}, inplace=True)

game_log_df.dropna(subset=["a_team", "h_team"], inplace=True)

game_log_df.head()

/Users/adityanaik/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (13,15,85,87) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,game_num,day_of_week,a_team,a_league,a_game_num,h_team,h_league,h_game_num,a_runs,...,152,153,154,155,156,157,158,159,160,161
0,2000-03-29,0.0,Wed,CHN,NL,1.0,NYM,NL,1.0,5.0,...,Todd Zeile,3.0,ordor001,Rey Ordonez,6.0,hampm001,Mike Hampton,1.0,NaN,Y
1,2000-03-30,0.0,Thu,NYN,NL,2.0,CHC,NL,2.0,5.0,...,Jeff Huson,6.0,giraj001,Joe Girardi,2.0,farnk001,Kyle Farnsworth,1.0,NaN,Y
2,2000-04-03,0.0,Mon,COL,NL,1.0,ATL,NL,1.0,0.0,...,Eddie Perez,2.0,weisw001,Walt Weiss,6.0,maddg002,Greg Maddux,1.0,NaN,Y
3,2000-04-03,0.0,Mon,MIL,NL,1.0,CIN,NL,1.0,3.0,...,Aaron Boone,5.0,tuckm001,Michael Tucker,7.0,harnp001,Pete Harnisch,1.0,NaN,Y
4,2000-04-03,0.0,Mon,SFN,NL,1.0,FLA,NL,1.0,4.0,...,Brant Brown,9.0,redmm001,Mike Redmond,2.0,ferna001,Alex Fernandez,1.0,NaN,Y


In [55]:
new_df = pd.merge(game_log_df, elo_df,  how='left', left_on=['date','a_team', 'h_team'], right_on = ['date','team2', 'team1'])
new_df.dropna(subset=["a_team", "h_team"], inplace=True)
new_df.head()

,date,game_num,day_of_week,a_team,a_league,a_game_num,h_team,h_league,h_game_num,a_runs,...,pitcher1_rgs,pitcher2_rgs,pitcher1_adj,pitcher2_adj,rating_prob1,rating_prob2,rating1_post,rating2_post,score1,score2
0,2000-03-29,0.0,Wed,CHN,NL,1.0,NYM,NL,1.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-03-30,0.0,Thu,NYN,NL,2.0,CHC,NL,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-04-03,0.0,Mon,COL,NL,1.0,ATL,NL,1.0,0.0,...,57.248,52.522,-0.189571,-0.015488,0.638840,0.361160,1552.802,1478.758,2.0,0.0
3,2000-04-03,0.0,Mon,MIL,NL,1.0,CIN,NL,1.0,3.0,...,54.857,48.899,-0.324579,0.178217,0.611602,0.388398,1534.164,1482.151,3.0,3.0
4,2000-04-03,0.0,Mon,SFN,NL,1.0,FLA,NL,1.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
unique_in_logs = game_log_df['h_team'].unique()

In [52]:
unique_in_elo = elo_df['team1'].unique()

In [53]:
for x in unique_in_logs:
    if x not in unique_in_elo:
        print(x)

In [54]:
for x in unique_in_elo:
    if x not in unique_in_logs:
        print(x)